# Scrapping data YouTube

## Menggunakan library selenium
Selenium digunakan untuk software untuk mengotomatisasi pengujian aplikasi web pada browser.
Dalam hal ini, Selenium digunakan untuk menavigasi ke halaman browser yang mengarah pada youtube untuk diambil dalam 

In [ ]:
! pip install selenium --quiet

## Menggunakan library NLTK (Natural Language Toolkit)
Digunakan untuk memproses teks Tokenisasi, Stemming.

In [ ]:
! pip install nltk

## Menginstal indoNLP


In [ ]:
! pip install indoNLP

     -------------------------------------- 121.9/121.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrapping data youtube

In [ ]:
# Menyimpan data yang akan di crawling
data = []

url = "https://www.youtube.com/watch?v=_HwJW5f5jFQ&ab_channel=NajwaShihab"

# Menghubungkan driver local ke link youtube yang dituju
with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:    
    wait = WebDriverWait(driver, 10)
    driver.get(
        url)

    # melakukan perulangan untuk scrolling data sebanyak 10 kali
    for item in range(10):
        wait.until(EC.visibility_of_element_located(
            (By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    # mencari elemen dengan tag content pada halaman youtube
    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21632\1192782035.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:


## Memasukkan ke dalam Data Frame

In [ ]:
import pandas as pd   
df = pd.DataFrame(data, columns=['komentar'])
df

,komentar
0,ID\nSkip navigation\nSign in\n0:00 / 1:15:25\n...
1,
2,Wawancara bagus dan fair. Dinamika masih akan ...
3,Dari Manado Renjana Tour pernah 3 hr full bers...
4,Rasanya Pak Ganjar ini upgrade dari Jokowi. (S...
...,...
144,
145,
146,
147,


## Cleaning Data

Memproses data kata yang tidak digunakan 

In [ ]:
import re
import string
import numpy as np

# Teks Cleaning


def cleaning(text):
    # Menghilangkan tag bawaan HTML

    text = re.compile(
        '<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # mengubah defaukt teks ke huruf kecil
    text = text.lower()

    # menghapus karakter kosong di depan dan belakang teks
    text = text.strip()

    # menghapus tanda baca, karakter spesial, dan spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # menghapus teks yang berisi angka
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    # Menghapus bawaan teks youtube
    text = re.sub('id', '', text)
    text = re.sub('skip', '', text)
    text = re.sub('navigation', '', text)
    text = re.sub('sign', '', text)

    return text

## Steamming data

Mengambil kata dasar



In [ ]:
# Import 
from nltk.stem import PorterStemmer
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang,
                emoji_to_words, remove_html])
# steamming
ps = PorterStemmer()

In [ ]:
df['hasil komentar'] = df['komentar'].apply(lambda x: pipe(x))
df['hasil komentar'] = df['hasil komentar'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['hasil komentar'] = df['hasil komentar'].replace('', np.nan)

## Cek nilai kosong



In [ ]:
print(df.isna().sum())

komentar          0
hasil komentar    8
dtype: int64


## Hapus nilai kosong

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

,komentar,hasil komentar
0,ID\nSkip navigation\nSign in\n0:00 / 1:15:25\n...,ini live eksklusif ganjar pranowo bicara c...
2,Wawancara bagus dan fair. Dinamika masih akan ...,wawancara bagus dan fair dinamika masih akan t...
3,Dari Manado Renjana Tour pernah 3 hr full bers...,dari manado renjana tour pernah hari ful bersa...
4,Rasanya Pak Ganjar ini upgrade dari Jokowi. (S...,rasanya pak ganjar ini upgrade dari jokowi sem...
5,Pak Ganjar memang oke \nApapun pertanyaan yg ...,pak ganjar memang oke apapun pertanyaan yang d...
...,...,...
137,"Alhamdulillah Pak Ganjar ,, semoga di berikan ...",alhamdulillah pak ganjar semoga di berikan kes...
138,"Masya allah cerdas banget sih Pak…,saya cerna ...",masya allah cerdas banget sih pak saya cerna j...
139,Sy bingung ngga nemu konsep dy bngun negara .....,saya bingung enggak nemu konsep dia bangun neg...
140,Cerdas nya pak ganjar mudah2an bisa melebihi p...,cerdas nya pak ganjar mudah mudahan bisa meleb...


## Import ke ke csv

Setelah mendapatkan hasilnya ubah ke dalam bentuk csv

In [ ]:
df.to_csv('scrapping_youtube.csv', index=False)